In [46]:
import pandas as pd
import requests
import pprint
from icecream import ic
import numpy as np
from difflib import get_close_matches

In [82]:
df = pd.read_excel('Municipios_Corrigir.xlsx')
df = df[df['ESTADO'].notnull()]
df = df[df['CIDADE'].notnull()]

In [44]:
print(df['ESTADO'].unique().size)

18


In [41]:
df.head()

,CIDADE,ESTADO
0,NaN,DF
1,AN?POLIS,GO
2,S¿O JOS¿ DO RIO PRETO,SP
3,S?O CAETANO DO SUL,SP
4,S?O JOS? DO RIO PRETO,SP


In [29]:
def get_brazilian_states():
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/estados"
    response = requests.get(url)
    
    if response.status_code == 200:
        states = response.json()
        return states
    else:
        raise Exception(f"Failed to retrieve data: {response.status_code}")

In [27]:
def create_state_dict(states):
    state_dict = {state['sigla']: state['id'] for state in states}
    return state_dict

In [28]:
states = get_brazilian_states()
state_dict = create_state_dict(states)

ic| response.status_code: 200
ic| 'teste'


In [47]:
def get_cities_by_state(state_id):
    url = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/{state_id}/municipios"
    response = requests.get(url)
    
    if response.status_code == 200:
        cities = response.json()
        return [city['nome'] for city in cities]
    else:
        raise Exception(f"Failed to retrieve data: {response.status_code}")

In [83]:
def fix_cities_names(df, state_dict):
    for sigla in df['ESTADO'].unique():
        state_id = state_dict.get(sigla)
        if state_id:
            city_names_api = get_cities_by_state(state_id)

            for index, row in df[df['ESTADO'] == sigla].iterrows():
                city_name = row['CIDADE']
                city_name = city_name.replace('?', '').title()
                city_name = city_name.replace('¿', '')
                
                closest_match = get_close_matches(city_name, city_names_api)
                if closest_match:
                    df.at[index, 'CIDADE_FIX'] = closest_match[0]
    
    return df

In [84]:
df = fix_cities_names(df, state_dict)

In [85]:
df.head(10)

,CIDADE,ESTADO,CIDADE_FIX
1,AN?POLIS,GO,Anápolis
2,S¿O JOS¿ DO RIO PRETO,SP,São José do Rio Preto
3,S?O CAETANO DO SUL,SP,São Caetano do Sul
4,S?O JOS? DO RIO PRETO,SP,São José do Rio Preto
5,TABO?O DA SERRA,SP,Taboão da Serra
6,JUNDIA?,SP,Jundiaí
7,GUARUJ¿,SP,Guarujá
8,SAO B.DO CAMPO,SP,São Bernardo do Campo
9,CAMB?,PR,Cambé
10,SERT?OZINHO,SP,Sertãozinho
